In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from datasets import Dataset

In [2]:
data_path = "/content/drive/MyDrive/ML_final_roberta/data_for_training.csv"

In [3]:
df = pd.read_csv(data_path)
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

In [4]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
label_encoder = LabelEncoder()
train_df['label_encoded'] = label_encoder.fit_transform(train_df['Category'])
val_df['label_encoded'] = label_encoder.transform(val_df['Category'])



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [5]:
def preprocess_function(examples):
    result = tokenizer(examples['Text'], truncation=True, padding='max_length', max_length=512)
    result['labels'] = examples['label_encoded']
    return result

In [6]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [7]:
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/11007 [00:00<?, ? examples/s]

Map:   0%|          | 0/1224 [00:00<?, ? examples/s]

In [20]:
num_labels = len(label_encoder.classes_)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
model = model.to(device)

აქ პარამეტრების მცირე მიმოხილვას გავაკეთებ. learning rate საკმაოდ სტანდარტული რამ ავირჩიე, არანაირი ლოგიკის მიხედვით არ წავსულვარ, უბრალოდ მიღებული პრაქტიკა. batch და accumulation steps compute-ის მიხედვით ავარჩიე (A100-ზე გავწვრთენი). warmup-იც საკმაოდ პირობითია. ვიფიქრე ნაკლების აღება, თუმცა დატის სიმცირის გამო 0.005-ის მაგივრად 0.05 ავიღე. lr_scheduler_type ორი ვარიანტი მქონდა. წრფივი და კოსინუსი. კარგი იქნებოდა რომ ორივე გამეტესტა იმიტომ რომ საბოლოოდ მაგას დიდი ზეგავლენა აქვს ტრენინგზე და LR-ზე, მაგრამ ამ ეტაპზე აღარ მაქვს compute unit-ები, რომ გავაგრძელო წვრთნა. weight decay სტანდარტული რეგულარიზაციის პარამეტრია რომელთანაც თამაში შეიძლება, მაგრამ ზოგადად მიღებულია რომ ან 0.0001 ან 0.001 ან დაახლოებით ამ არეალში იღებენ, ესეც აბსოლუტურად პირობითია. დანარჩენი უკვე უბრალოდ ლოგირებისა და სეივინგის პარამეტრებია რაც არ უნდა იყოს რთული გასაგები, ფაქტობრივად self-explanatory არის.


In [24]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/ML_final_roberta/output_dir',
    learning_rate=1e-4,
    num_train_epochs=30,
    per_device_train_batch_size=48,
    per_device_eval_batch_size=48,
    warmup_ratio=0.05,
    lr_scheduler_type = 'linear',
    weight_decay=0.0001,
    gradient_accumulation_steps=32,
    evaluation_strategy="steps",
    eval_steps = 5,
    logging_steps = 5,
    logging_strategy = "steps",
    save_strategy="steps",
    save_steps=35,
    logging_dir='/content/drive/MyDrive/ML_final_roberta/logging_dir',
    load_best_model_at_end=True,
    metric_for_best_model="loss"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [26]:
trainer.train()

Step,Training Loss,Validation Loss
5,3.874800,3.823989
10,3.756600,3.557616
15,3.344800,2.922512
20,2.750100,2.299869
25,2.155100,1.755197
30,1.650000,1.360181
35,1.276500,1.036294
40,1.007600,0.797620
45,0.834900,0.669432
50,0.643100,0.577083


KeyboardInterrupt: 

ეს განზრახ გავაჩერე, რადგან გაპლატოებას იწყებდა. ვალიდაციის ლოსმა დაიწყო ზემოთ სვლა.

ლოგიკურად, 75-80 სტეპი უნდა იყოს ყველაზე სწორი არჩევანი აქ. მერე, ვალიდაციის ლოსი გაპლატოვებას იწყებს. ჩვენ დავასეივეთ 70-ე ჩეკპოინტის მოდელი, ამიტომ ეგეც წესით "წავა". თუ არადა, 5 დამატებით სტეპზე შეგვიძლია გავწვრთნათ. თუმცა არ მგონია საჭირო იყოს.

In [28]:
model = RobertaForSequenceClassification.from_pretrained("/content/drive/MyDrive/ML_final_roberta/output_dir/checkpoint-70")


აქ მინდა რომ პირველი 3 ფრედიქშენი და მაგათი confidence ვნახო.

In [41]:
def predict_texts(input_texts,top_k=3):
    encoded_inputs = tokenizer(input_texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**encoded_inputs)

    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1)

    top_probs, top_preds = torch.topk(probabilities, k=top_k, dim=1)
    top_probs = top_probs.numpy()
    top_preds = top_preds.numpy()

    results = []
    for i in range(top_preds.shape[0]):
        labels = label_encoder.inverse_transform(top_preds[i])
        confidences = top_probs[i]
        results.append(list(zip(labels, confidences)))

    return results

In [44]:
input_texts = []
for i in range(0,3):
  CV_path = f"/content/drive/MyDrive/ML_final_roberta/CV{i}.txt"
  with open(CV_path, "r") as f:
    txt = f.read()
    input_texts.append(txt)

In [45]:
predict_texts(input_texts)

[[('Data Science', 0.8852416),
  ('Testing', 0.015259309),
  ('Education', 0.013214436)],
 [('Data Science', 0.29401797),
  ('Testing', 0.21805684),
  ('ETL Developer', 0.061151262)],
 [('ETL Developer', 0.20625137),
  ('Python Developer', 0.119231805),
  ('DotNet Developer', 0.07534327)]]

პირველი არის ჩემი სივი, რასაც ძალიან მარტივად მიხვდა. მეორე და მესამე შედარებით დამაბნეველი სივიებია, რომლებიც ჩემს მეგობრებს ეკუთვნის. მეორე განსაკუთრებით, თუმცა მაინც საინტერესოა რატომ ვერ ხვდება მესამეს საერთოდ. ETL საკმაოდ ლოგიკურია, თუმცა data science-ს ველოდებოდი. ძალიან ბევრი რამ უწერია დატა ინჟინერიაზე, შესაძლებელია ეგ იყოს მიზეზი.

In [46]:
def get_predictions(dataset):
    trainer = Trainer(model=model, tokenizer=tokenizer)
    raw_pred, _, _ = trainer.predict(dataset)
    predictions = torch.argmax(torch.tensor(raw_pred), dim=1)
    return predictions.numpy()

In [49]:
true_labels = val_dataset['labels']
predictions = get_predictions(val_dataset)

accuracy = accuracy_score(true_labels, predictions)
print(f"Accuracy on the validation set: {accuracy:.4f}")

Accuracy on the validation set: 0.9118


# საბოლოოდ, ვფიქრობ ეს ბევრად უკეთესი მიდგომაა ვიდრე სტატისტიკური მოდელები და მე მგონია, რომ ეს შედეგებმაც აჩვენა. 0.9118 არის აშკარად უფრო მეტი, ვიდრე ჩვენ მივიღეთ TF-IDF-ით და შემდეგ სტატისტიკური, კლასიკური ML-ის მოდელებით, რაც ნუ, ლოგიკურია, რადგან ეს არქიტექტურა სწორედ ასეთი ტასკებისთვისაა შექმნილი